In [1]:
from typing import Callable

import torch
import numpy
import matplotlib.pyplot as plt

from utilities import CostParameters, least_square_cost, ridge_cost

In [2]:
torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    torch.set_default_device("cuda")

In [3]:
TEST_DATA_SIZE = 100
X = numpy.sort(numpy.random.rand(TEST_DATA_SIZE, 1), axis=0)
Y = numpy.random.rand(TEST_DATA_SIZE, 1)

In [4]:
input_layer = 1
output_layer = 1
learning_rate = 1e-5

def train(hidden_layer: int, epochs: int, cost_func: Callable[[CostParameters], torch.Tensor], cost_func_name: str, print_debug_info: bool = False) -> None:
    x_tensor = torch.tensor(X)
    y_tensor = torch.tensor(Y)

    weights_layer1 = torch.rand(input_layer, hidden_layer, requires_grad=True)
    biases_layer1 = torch.rand(hidden_layer, requires_grad=True)
    weights_layer2 = torch.rand(hidden_layer, output_layer, requires_grad=True)
    biases_layer2 = torch.rand(output_layer, requires_grad=True)

    def predict() -> torch.Tensor:
        return (x_tensor @ weights_layer1 + biases_layer1) @ weights_layer2 + biases_layer2

    for i in range(epochs):
        predicted = predict()

        cost_params = CostParameters(y_tensor, predicted, [weights_layer1, weights_layer2], [biases_layer1, biases_layer2])
        loss = cost_func(cost_params)

        if print_debug_info:
            iteration_id = i + 1
            if iteration_id % 500 == 0:
                print(f"Info at epoch #{iteration_id}:")
                print(f"Loss: {loss.item()}")

        loss.backward()

        with torch.no_grad():
            weights_layer1 -= learning_rate * weights_layer1.grad
            biases_layer1 -= learning_rate * biases_layer1.grad
            weights_layer2 -= learning_rate * weights_layer2.grad
            biases_layer2 -= learning_rate * biases_layer2.grad

            weights_layer1.grad = None
            biases_layer1.grad = None
            weights_layer2.grad = None
            biases_layer2.grad = None

    final_prediction = numpy.array(predict().detach().cpu())

    plt.title(f"Training a 2D regression model with {hidden_layer} neurons within the hidden layer after {epochs} epochs")
    plt.scatter(X, Y)
    plt.plot(X, final_prediction)

    plt.savefig(f"plots/linear_regression_2d_{hidden_layer}_{epochs}_{cost_func_name}.png", bbox_inches='tight')
    plt.close()

In [5]:
train(1, 5_000, least_square_cost, "lsc", print_debug_info=True)

Info at epoch #500:
Loss: 10.080791952455073
Info at epoch #1000:
Loss: 10.016561102175556
Info at epoch #1500:
Loss: 9.965180723536829
Info at epoch #2000:
Loss: 9.918633781585136
Info at epoch #2500:
Loss: 9.876210741415413
Info at epoch #3000:
Loss: 9.837432131664936
Info at epoch #3500:
Loss: 9.801888483083616
Info at epoch #4000:
Loss: 9.7692263981695
Info at epoch #4500:
Loss: 9.739139550172796
Info at epoch #5000:
Loss: 9.711361347036622


![Linear regression 2D model with 1 hidden neuron and 5000 epochs](./plots/linear_regression_2d_1_5000_lsc.png)

In [6]:
train(10, 5_000, least_square_cost, "lsc", print_debug_info=True)

Info at epoch #500:
Loss: 10.7202007062715
Info at epoch #1000:
Loss: 10.246823283458669
Info at epoch #1500:
Loss: 9.919855997285646
Info at epoch #2000:
Loss: 9.693683593699967
Info at epoch #2500:
Loss: 9.537242290114111
Info at epoch #3000:
Loss: 9.429133364813875
Info at epoch #3500:
Loss: 9.354527330259593
Info at epoch #4000:
Loss: 9.303121992168268
Info at epoch #4500:
Loss: 9.26775852671381
Info at epoch #5000:
Loss: 9.24346752067375


![Linear regression 2D model with 10 hidden neuron and 5000 epochs](./plots/linear_regression_2d_10_5000_lsc.png)

In [7]:
train(100, 5_000, least_square_cost, "lsc", print_debug_info=True)

Info at epoch #500:
Loss: 12.352032354564283
Info at epoch #1000:
Loss: 9.312027855239267
Info at epoch #1500:
Loss: 9.195409519734492
Info at epoch #2000:
Loss: 9.190939667449081
Info at epoch #2500:
Loss: 9.190768415499107
Info at epoch #3000:
Loss: 9.190761854989692
Info at epoch #3500:
Loss: 9.190761603667088
Info at epoch #4000:
Loss: 9.190761594039362
Info at epoch #4500:
Loss: 9.190761593670539
Info at epoch #5000:
Loss: 9.190761593656413


![Linear regression 2D model with 100 hidden neuron and 5000 epochs](./plots/linear_regression_2d_100_5000_lsc.png)

In [8]:
train(1, 5_000, ridge_cost(least_square_cost, 0.5), "ridge+lsc", print_debug_info=True)

Info at epoch #500:
Loss: 10.100892868166715
Info at epoch #1000:
Loss: 9.543229674925461
Info at epoch #1500:
Loss: 9.508770388422429
Info at epoch #2000:
Loss: 9.501523771441823
Info at epoch #2500:
Loss: 9.495882050413554
Info at epoch #3000:
Loss: 9.490499702423914
Info at epoch #3500:
Loss: 9.48529686760664
Info at epoch #4000:
Loss: 9.480260257525849
Info at epoch #4500:
Loss: 9.47560277379135
Info at epoch #5000:
Loss: 9.47393818643749


![Linear regression 2D model with 1 hidden neuron and 5000 epochs](./plots/linear_regression_2d_1_5000_ridge+lsc.png)

In [9]:
train(10, 5_000, ridge_cost(least_square_cost, 0.5), "ridge+lsc", print_debug_info=True)

Info at epoch #500:
Loss: 12.489515030353527
Info at epoch #1000:
Loss: 11.7808091618845
Info at epoch #1500:
Loss: 11.356283078722818
Info at epoch #2000:
Loss: 11.099797492431005
Info at epoch #2500:
Loss: 10.94378993693739
Info at epoch #3000:
Loss: 10.84825997540173
Info at epoch #3500:
Loss: 10.789277232137055
Info at epoch #4000:
Loss: 10.752436505781827
Info at epoch #4500:
Loss: 10.72903395284979
Info at epoch #5000:
Loss: 10.713798126177531


![Linear regression 2D model with 10 hidden neuron and 5000 epochs](./plots/linear_regression_2d_10_5000_ridge+lsc.png)

In [10]:
train(100, 5_000, ridge_cost(least_square_cost, 0.5), "ridge+lsc", print_debug_info=True)

Info at epoch #500:
Loss: 14.307892696231429
Info at epoch #1000:
Loss: 13.820984962128295
Info at epoch #1500:
Loss: 13.809672092232843
Info at epoch #2000:
Loss: 13.807011868720537
Info at epoch #2500:
Loss: 13.804509618641731
Info at epoch #3000:
Loss: 13.802010265322103
Info at epoch #3500:
Loss: 13.799510966483803
Info at epoch #4000:
Loss: 13.797011669865586
Info at epoch #4500:
Loss: 13.794512374511683
Info at epoch #5000:
Loss: 13.792013080408069


![Linear regression 2D model with 100 hidden neuron and 5000 epochs](./plots/linear_regression_2d_100_5000_ridge+lsc.png)